In [86]:
# Import packages
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.cm  import tab20
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap
import matplotlib.dates as mdates
import matplotlib.patches as mpatches

from ipywidgets import interact, widgets, Dropdown
from IPython.display import display, Markdown
import time
import random

In [54]:
# F1 DATA API 
import fastf1
import fastf1.plotting
fastf1.plotting.setup_mpl();
colormap = mpl.cm.RdYlGn

# F1 Data Analysis Tool
#### Developed By: Hunter Sprigings

##### Thank you for taking the time to explore my program. This tool is intended to assist users in analyzing post-race F1 data, offering a deeper insight into individual and team performance across races and laps. For more thorough insights, it is advisable to use this data alongside visual race recordings. Unfortunately, the API on which this code is dependant on will be deprecated after 2024, so please enjoy it while it lasts. If you have any questions or feedback please reach out I, am always looking to improve :)

## LOAD IN TRACK DATA & RUN

##### To load your data, please select a date and year. Allow some time for the program to retrieve the data. 

###### Notes:
###### - If experiencing misalignment of chart widths, reduce browser size (Cmd|Ctr +). These are not dynamically set to the browswer width.
###### - Multiple drivers can be selected at a time by holding Cmd|Ctr + Click


In [55]:
# Creates selection for user to select race & year
years = [i for i in range(2005, 2024)]
dropdown = Dropdown(options=years, description='Select a Year to Analyze:')
def on_dropdown_change(change):
    global selected_year
    selected_year = change.new
    print("Selected option:", change.new)
    
dropdown.observe(on_dropdown_change, names='value')

In [56]:
#Get race names for dropdown
pd.options.mode.chained_assignment = None  # default='warn'
schedule = fastf1.get_event_schedule(2024)
event_names = schedule['EventName']

In [57]:
import logging
logging.getLogger('fastf1').setLevel(logging.WARNING)
warnings.filterwarnings('ignore')

In [85]:
# Create progress bar widget
progress_bar = widgets.IntProgress(
    min=0,
    max=100,
    description='Loading:',
    bar_style='', 
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)

display(progress_bar)

def load_lap_data(race_year, event_name):
    try:
        #Progress bar animation
        progress_bar.value = 25
        time.sleep(2.5)
        progress_bar.value = 50
        time.sleep(2.5)
        progress_bar.value = 75
        
        # Attempt to load lap data
        session = fastf1.get_session(race_year, event_name, 'R')
        session.load(laps=True)
        race = session.laps
        race['race_name'] = event_name
        
        progress_bar.value = 100
            
            
        #Call the graphs for plotting
        plot_driver_finishes_interactive(race)
        speed_boxplot_interactive(race)
        lap_speeds_interactive(race)
        lap_times_interactive(race,session)
        
    
    except fastf1.core.DataNotLoadedError:
        progress_bar.value = 0
        print("ERROR: Your Selection Is Invalid. Please Try Selecting a Different Race or Year")
        
    return 
        

# Interactive load
years = widgets.IntSlider(min=2015, max=2024, step=1, value=2024, description='Year:')
race = interact(load_lap_data, race_year=years, event_name=widgets.Dropdown(options=event_names, value='Canadian Grand Prix'))

IntProgress(value=0, description='Loading:', style=ProgressStyle(bar_color='maroon'))

interactive(children=(IntSlider(value=2024, description='Year:', max=2024, min=2015), Dropdown(description='ev…

In [59]:
#Helper function to get drivers colours 
def get_color(driver):
            try:
                color = fastf1.plotting.driver_color(driver)    
            except:
                # Retrieves random colour if driver is not in the current year
                color = (100, random.randint(0, 255), random.randint(0, 255))
                color = '#%02x%02x%02x' % color
            return color

In [1]:
def plot_driver_finishes_interactive(race):

    #Interactive wrapper
    @interact(driver_list=widgets.SelectMultiple(
        options=race['Driver'].unique(),
        value=(race['Driver'].unique()[0],),
        description='Driver Name'))
    
    
    #Plot race positions throughout
    def plot_drivers(driver_list):
        plt.figure(figsize=(20, 7))
        plt.xticks(np.arange(1, race['LapNumber'].max(), 1))
        plt.yticks(np.arange(1, len(race['Driver'].unique()), 1))
        plt.grid()
        plt.xlim(0, race['LapNumber'].max())  
        plt.ylim(0, len(race['Driver'].unique())) 
        plt.gca().invert_yaxis()

        # Plot each selected driver with a unique color
        for i, driver in enumerate(driver_list):
            drivers_speed = race[race['Driver'] == driver]
            color = get_color(driver)

            sns.lineplot(data=drivers_speed, x='LapNumber', y='Position', linewidth=3, label=driver, color=color)
        
        legend = plt.legend(loc='center', bbox_to_anchor=(0.5, -0.2), ncol=10)
        plt.title('Driver Places By Lap')

        plt.show()

In [95]:
def speed_boxplot_interactive(race):
    
    #Interactive wrapper
    @interact(driver_list=widgets.SelectMultiple(
        options=race['Driver'].unique(),
        value=(race['Driver'].unique()[0],),
        description='Driver Name'))
    
    #Plot speed boxplot
    def plot_driver_speeds(driver_list):
        
        # Each selected driver with a unique colour
        driver_colours = {}
        driver_list = list(driver_list)
        for driver in driver_list:
            color = get_color(driver)
            driver_colours[driver] = color
            

        try:
    
            # Only get the fast laps to avoid some outlier datapoints
            driver_laps = race.pick_quicklaps()
            driver_laps = driver_laps[driver_laps['Driver'].isin(driver_list)]
            driver_laps = driver_laps.reset_index()

            # Seaborn doesn't have proper timedelta support so we have to convert timedelta to float 
            driver_laps["LapTime(s)"] = driver_laps["LapTime"].dt.total_seconds()
            
            #Order the team from the fastest to slowest
            driver_order = (
                driver_laps[["Driver", "LapTime(s)"]]
                .groupby("Driver")
                .median()["LapTime(s)"]
                .sort_values()
                .index
            )
            

            # plot
            fig, ax = plt.subplots(figsize=(20, 7))
            sns.boxplot(data=driver_laps,
                               x='Driver',
                               y='LapTime(s)',
                               order=driver_order,
                               palette=driver_colours,
                               whiskerprops=dict(color="white"),
                               boxprops=dict(edgecolor="white"),
                               medianprops=dict(color="white"),
                               capprops=dict(color="white"),
                               width = 0.75
                               )


            ax.set_ylabel("Lap Time (s)")
            plt.title('Driver Speed Comparisons')
            sns.despine(left=True, bottom=True)
            plt.show()
            
        except:
            print('DRIVER NOT IN RACE TRY ANOTHER SELECTION')
            
    

In [83]:
def lap_speeds_interactive(race):
    
    # Interactive wrapper
    @interact(driver_list=widgets.SelectMultiple(
        options=race['Driver'].unique(),
        value=(race['Driver'].unique()[0],),
        description='Driver Name'))
    
    # Plot speed laptimes
    def compare_driver_speeds(driver_list):
        
        # Mask data for only drivers selected
        selected_drivers = race[race['Driver'].isin(driver_list)]
        
        # Convert timedelta to float (in seconds)
        selected_drivers.loc[:, "LapTime(s)"] = selected_drivers["LapTime"].dt.total_seconds()
        
        fig, ax = plt.subplots(figsize=(20, 5))
        
        # Plot each driver's lap times
        for driver in driver_list:
            driver_laps = selected_drivers[selected_drivers['Driver'] == driver].reset_index()
            full_laps_index = range(0, len(race))
            
            # Reindex the df with the correct number of laps 
            df_reindexed = driver_laps.reindex(full_laps_index).reset_index()
            
            # Interpolate the missing time to allow for plotting
            df_reindexed['LapTime(s)'] = df_reindexed['LapTime(s)'].interpolate()
            
            color = get_color(driver)
            sns.lineplot(data=df_reindexed, x='LapNumber', y="LapTime(s)", color=color,linewidth=3, label=driver, ax=ax)
            ax.grid(visible=False)
            ax.set_xticks(np.arange(0, df_reindexed['LapNumber'].max() + 1, step=5))

        # Set labels and title
        ax.set_xlabel('Lap Number')
        ax.set_ylabel('Lap Time (s)')
        plt.title('Driver Lap Times & Tire Compounds')
        legend = plt.legend(loc='center', bbox_to_anchor=(0.5, -0.2), ncol=10)

        # Plot tire compounds
        fig_num = len(driver_list)
        fig, axes = plt.subplots(nrows=fig_num, ncols=1, figsize=(20, 1 * fig_num))  # Adjust height for consistency
        
        # Ensure axes is always a list for consistent handling
        if fig_num == 1:
            axes = [axes]
        
        for i, driver in enumerate(driver_list):
            driver_data = selected_drivers[selected_drivers['Driver'] == driver].copy()
            
            # Group by Compound and count LapNumbers
            driver_data['Compound'] = pd.Categorical(driver_data['Compound'], categories=driver_data['Compound'].unique(), ordered=True)
            grouped = driver_data.groupby(['Driver', 'Compound'])['LapNumber'].count().unstack().reset_index()
            compound_color = {'MEDIUM': 'yellow', 'HARD': 'white', 'SOFT': 'red', 'INTERMEDIATE': 'green', 'WET': 'blue'}
            
            # Plot the grouped data
            grouped.plot(kind='barh', stacked=True, color=[compound_color.get(x, 'grey') for x in grouped.columns[1:]], ax=axes[i])
            axes[i].set_xticks([])
            axes[i].set_yticks([])
            axes[i].set_ylabel(driver)
            patches = [mpatches.Patch(color=color, label=compound) for compound, color in compound_color.items()]

            # Plot only one legend     
            if i == (len(driver_list) - 1):
                axes[i].legend(handles=patches, title='Compound', loc='center', bbox_to_anchor=(0.5, -1), ncol=len(patches))
            else:
                axes[i].legend().set_visible(False)
                
        plt.show()


In [93]:
def lap_times_interactive(race,session):
    #Interactive dropdown
    driver_names = race['Driver'].unique()
    lap_numbers = race['LapNumber'].unique()

    #Interactive wrapper
    @interact(lap=widgets.IntSlider(min=lap_numbers.min(), max=lap_numbers.max(), step=1, value=lap_numbers.min(), description='Lap:'),
              driver1=widgets.Dropdown(options=driver_names, description='Driver 1:'),
              driver2=widgets.Dropdown(options=driver_names, description='Driver 2:'))
    
    def show_lap_times(lap, driver1, driver2):
        lap_data1 = race[(race['LapNumber'] == lap) & (race['Driver'] == driver1)]
        lap_data2 = race[(race['LapNumber'] == lap) & (race['Driver'] == driver2)]
        
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7), sharex=True, sharey=True)
        plt.title('Driver Direct Lap Time Comparison')
        
        #First Driver Track Data
        x = lap_data1.telemetry['X']            
        y = lap_data1.telemetry['Y']              
        color = lap_data1.telemetry['Speed'] 
        
        ax1.plot(lap_data1.telemetry['X'], lap_data1.telemetry['Y'], color='black', linestyle='-', linewidth=16, zorder=0)
        ax1.axis('off')
        ax1.scatter(x, y, c=color, cmap=mpl.cm.plasma)
        ax1.set_title(driver1)
        
        #Colour bar
        cbar = fig.colorbar(ax1.collections[0], ax=ax1, orientation='horizontal')
        cbar.set_label('Speed KM')
        
        
        #Second Driver Speed Chart
        x = lap_data2.telemetry['X']            
        y = lap_data2.telemetry['Y']              
        color = lap_data2.telemetry['Speed'] 
        
        
        ax2.plot(x, y, color='black', linestyle='-', linewidth=16, zorder=0)
        ax2.axis('off')
        ax2.scatter(x, y, c=color, cmap=mpl.cm.plasma)
        ax2.set_title(driver2)
        
        #Colour bar
        cbar = fig.colorbar(ax1.collections[0], ax=ax2, orientation='horizontal')
        cbar.set_label('Speed KM')
        
        
        # Plot actual speed data
        car_data = lap_data1.get_car_data().add_distance()
        car_data2 = lap_data2.get_car_data().add_distance()
        circuit_info = session.get_circuit_info()
        
        fig, ax = plt.subplots(figsize=(20, 3))
        driver1_color = get_color(driver1)
        driver2_color = get_color(driver2)
        ax.plot(car_data['Distance'], car_data['Speed'], color=driver1_color,label=driver1)
        ax.plot(car_data2['Distance'], car_data2['Speed'], color=driver2_color,label=driver2)

        # Minimum speed to slightly above the maximum speed.
        v_min = min(car_data['Speed'].min(), car_data2['Speed'].min())
        v_max = max(car_data['Speed'].max(), car_data2['Speed'].max())
        ax.vlines(x=circuit_info.corners['Distance'], ymin=v_min-20, ymax=v_max+20,
                  linestyles='dotted', colors='grey')
        
        # Plot corners on graph
        for _, corner in circuit_info.corners.iterrows():
            txt = f"{corner['Number']}{corner['Letter']}"
            ax.text(corner['Distance'], v_min-30, txt,
                    va='center_baseline', ha='center', size='small')

        ax.set_xlabel('Distance in m')
        ax.set_ylabel('Speed in km/h')
        ax.set_ylim([v_min - 40, v_max + 20])
        ax.legend()

        plt.show()